In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data.dataset import TensorDataset
import torch.optim as optim 
import torchvision
import torch.nn as nn

from torch.utils.data import DataLoader
from torch.utils.data import Dataset

In [2]:
#load deepdataset 
from google.colab import drive
drive.mount('/content/drive')

xtrain = pd.read_csv('/content/drive/My Drive/xtrain_32_10000.csv', header=None)
ytrain = pd.read_csv('/content/drive/My Drive/ytrain_32_10000.csv', header=None)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# data preprocessing
xtrain = np.asarray(xtrain).astype(np.float16)
ytrain = np.asarray(ytrain).astype(np.float16)

#xtrain= F.normalize(torch.Tensor(xtrain))
#ytrain= F.normalize(torch.Tensor(ytrain))

xtrain= torch.Tensor(xtrain)
ytrain= torch.Tensor(ytrain)

In [4]:
print('The number of elements on IRS is' + str(xtrain.shape[1]) + '\n The trainng data size is' + str(xtrain.shape[0]))

The number of elements on IRS is1024
 The trainng data size is10000


In [5]:
def get_device():
    if torch.cuda.is_available():
        device = 'cuda:0'
    else:
        device = 'cpu'
    return device
device = get_device()

In [6]:
# define training dataset
train_data = TensorDataset(xtrain, ytrain)
trainloader = DataLoader(train_data, batch_size=500, shuffle=True)

# number of the elements (i.e. features)
M = xtrain.shape[1]

In [7]:
# define the neural net class
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(in_features=M, out_features=2*M)
        self.dp1 = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(in_features=2*M, out_features=4*M)
        self.dp2 = nn.Dropout(p=0.5)
        self.fc3 = nn.Linear(in_features=4*M, out_features=4*M)
        self.dp3 = nn.Dropout(p=0.5)
        self.fc4 = nn.Linear(in_features=4*M, out_features=M)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dp1(x)
        x = F.relu(self.fc2(x))
        x = self.dp2(x)
        x = F.relu(self.fc3(x))
        x = self.dp3(x)
        x = self.fc4(x)
        return x

net = Net().to(device)
print(net)

Net(
  (fc1): Linear(in_features=1024, out_features=2048, bias=True)
  (dp1): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=2048, out_features=4096, bias=True)
  (dp2): Dropout(p=0.5, inplace=False)
  (fc3): Linear(in_features=4096, out_features=4096, bias=True)
  (dp3): Dropout(p=0.5, inplace=False)
  (fc4): Linear(in_features=4096, out_features=1024, bias=True)
)


In [8]:
# loss
criterion = nn.MSELoss()
# optimizer
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
#epochs
epochs = 20

In [9]:
def train(net, trainloader):
    for epoch in range(epochs): # no. of epochs
        running_loss = 0
        for data in trainloader:
            # data to GPU if available
            features, targets = data[0], data[1]

            features = features.to(device)
            targets = targets.to(device)

            # set the parameter gradients to zero
            optimizer.zero_grad()
            outputs = net(features)
            loss = criterion(outputs, targets)

            # propagate the loss backward
            loss.backward()
            # update the gradients
            optimizer.step()
 
            running_loss += loss.item()
        print('[Epoch %d] loss: %.3f' %
                      (epoch + 1, running_loss/len(trainloader)))
 
    print('Done Training')


In [10]:
train(net, trainloader)

[Epoch 1] loss: 0.018
[Epoch 2] loss: 0.015
[Epoch 3] loss: 0.013
[Epoch 4] loss: 0.012
[Epoch 5] loss: 0.011
[Epoch 6] loss: 0.010
[Epoch 7] loss: 0.010
[Epoch 8] loss: 0.009
[Epoch 9] loss: 0.009
[Epoch 10] loss: 0.009
[Epoch 11] loss: 0.009
[Epoch 12] loss: 0.009
[Epoch 13] loss: 0.009
[Epoch 14] loss: 0.009
[Epoch 15] loss: 0.009
[Epoch 16] loss: 0.009
[Epoch 17] loss: 0.009
[Epoch 18] loss: 0.009
[Epoch 19] loss: 0.009
[Epoch 20] loss: 0.009
Done Training
